## License Plate Recognition based on image processing and OCR
Ref - https://www.pyimagesearch.com/2020/09/21/opencv-automatic-license-number-plate-recognition-anpr-with-python/

### Algorithm Flow
1. Detect and localize a license plate in an input image/frame based on image processing 
2. Extract the characters from the license plate
3. Apply Optical Character Recognition (OCR) to recognize the extracted characters

### Import

In [28]:
import cv2
import imutils
import numpy as np

from easyocr import Reader
from matplotlib import pyplot as plt
from skimage.segmentation import clear_border

### Parameter

In [29]:
IMG_FPATH = 'P9180007.jpg'

In [30]:
# Range of license plate aspect ratio
MIN_AR, MAX_AR = 2, 6

# Morphology Operation Parameters
MORPH_KER_SHAPE = cv2.MORPH_RECT
MORPH_KER_SIZE = (13, 5)
MORPH_OP1 = cv2.MORPH_BLACKHAT
MORPH_OP2 = cv2.MORPH_CLOSE

# Gaussian Blur Kernel Size
BLUR_KER_SIZE = (5, 5)

# Maximum number of detection candidate
NUM_KEEP = 5

In [31]:
# EasyOCR Lang
LANGS = ['en']

### Load Data

In [32]:
img = cv2.imread(IMG_FPATH)

### Preprocessing

In [33]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

### License Plate Detection

#### 1. Find Text Area

In [34]:
# Apply Black-Hat Morphology Operation => Make dark text over light background more obvious
# Difference between original image and image after morphology closing
morph_ker = cv2.getStructuringElement(MORPH_KER_SHAPE, MORPH_KER_SIZE)
gray_morph = cv2.morphologyEx(gray, MORPH_OP1, morph_ker)

In [35]:
# Compute gradient image along x-direction only and normalize
gray_gradx = cv2.Sobel(gray_morph, cv2.CV_32F, dx=1, dy=0, ksize=3)
gray_gradx = np.abs(gray_gradx)
gray_gradx = 255 * (gray_gradx - gray_gradx.min()) / (gray_gradx.max() - gray_gradx.min())
gray_gradx = gray_gradx.astype(np.uint8)

In [36]:
# Find text areas by adaptive threshold (Apply blurring and closing first to fill small holes)
gray_gradx_blur = cv2.GaussianBlur(gray_gradx, BLUR_KER_SIZE, 0)
gray_gradx_blur = cv2.morphologyEx(gray_gradx_blur, MORPH_OP2, morph_ker)
text_mask = cv2.threshold(gray_gradx_blur, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

#### 2. Find Bright Area (Assume license plate background is white)

In [37]:
# Find bright areas by adaptive threshold (Apply closing first to fill small holes)
# Pixel > OTSU Threshold = MaxVal (255)
gray_morph2 = cv2.morphologyEx(gray, MORPH_OP2, morph_ker)
bright_mask = cv2.threshold(gray_morph2, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

#### 3. Get mask of possible license plate area

In [38]:
# Multiple text mask and bright mask
# Perform series of erosions and dilations to denoise
mask = cv2.bitwise_and(text_mask, text_mask, mask=bright_mask)
mask = cv2.dilate(mask, None, iterations=1)
mask = cv2.erode(mask, None, iterations=1)

#### 4. Extract possible license plate area

In [39]:
# Find contour with top-K large area (Use element 0 for OpenCV4. Use element 1 for OpenCV3)
cnts = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:NUM_KEEP]

In [40]:
# Apply geometry contraint
lpcnt, lproi, lpbox = None, None, None

for cnt in cnts:
    # Get bounding rectangle of each contour
    x, y, w, h = cv2.boundingRect(cnt)

    # Aspect ratio check
    ar = w / float(h)
    if ar >= MIN_AR and ar <= MAX_AR:
        lpbox = (x, y, w, h)
        lpcnt = cnt

        # Extract binarized ROI
        lproi = gray[y:y+h, x:x+w]
        lproi = cv2.threshold(lproi, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

        break

### OCR

In [41]:
# Initialize EasyOCR
reader = Reader(LANGS)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [42]:
# OCR
# List of (box coordinate list in (x, y), Text, Probability)
results = reader.readtext(lproi)

### Visualization

In [43]:
img_vis = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
for bbox, text, prob in results:
    # unpack bbox
    tl, tr, br, bl = bbox
    tl = (int(tl[0] + lpbox[0]), int(tl[1] + lpbox[1]))
    br = (int(br[0] + lpbox[0]), int(br[1] + lpbox[1]))

    # Overlay
    cv2.rectangle(img_vis, tl, br, (0, 255, 0), 2)
    cv2.putText(img_vis, text, (tl[0], tl[1] - 10), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 1)

cv2.namedWindow('Result', cv2.WINDOW_NORMAL)
cv2.imshow('Result', img_vis)
cv2.waitKey(0)
cv2.destroyAllWindows()